In [ ]:
import os
from glob import glob
import tifffile as tif
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
base_path="D:/Ashish_GWL/Full_Dataset/DATA4/"
year_path=glob(base_path+"*")
row,col,factor,month=44,64,4,6

In [ ]:
def factor_preprocessing(year):
    k=0
    factor=np.empty([row,col,4,month])
    img_names=glob(year+"/PrePost/GRACE/*.tif")
    
    for img_name in img_names:
        img_name=os.path.split(img_name)[-1]
        #print(img_name)
        grace_img=data_filtering(np.array(tif.imread(year+"/PrePost/Grace/"+img_name)),-200,200,210)
        rainfall=data_filtering(np.array(tif.imread(year+"/PrePost/Rainfall/"+img_name)),0,250,260)
        soil_moisture=data_filtering(np.array(tif.imread(year+"/PrePost/SM/"+img_name)),0,200,210)
        ET=data_filtering(np.array(tif.imread(year+"/PrePost/ET/"+img_name)),0,200,210)
        #run_off=data_filtering(np.array(tif.imread(year+"/PrePost/Runoff/"+img_name)),0,200,210)
        #print("Grace",grace_img.max(),grace_img.min())
        #print("rainfall", rainfall.max(),rainfall.min())
        factor[:,:,0,k]=grace_img[:row,:col]
        factor[:,:,1,k]=rainfall[:row,:col]
        factor[:,:,2,k]=soil_moisture[:row,:col]
        factor[:,:,3,k]=ET[:row,:col]
        #factor[:,:,4,k]=run_off[:row,:col]
        k=k+1
        #print(factor[:,:,0,k].max(),factor[:,:,0,k].min())
    return factor         
def data_filtering(img,low,high,value):
    temp=img
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if (img[i,j]>low and img[i,j]<high):
                temp[i,j]=img[i,j]
            else:
                temp[i,j]=value
    return temp/value
    #return (temp-temp.min()/temp.max()-temp.min())
    

In [ ]:
yr=0
data_set=np.empty([12,44,64,4,6]) # (12 is years, size of image , 4 is factor, 6 is month cycle )
for year in year_path:
    montly=factor_preprocessing(year)
    print(year)
    data_set[yr]=montly
    yr=yr+1
print("Image sequence complete !!!!!")

Image lable (End-result)

In [ ]:
label_year=glob(base_path+"*")
label_arr=np.empty([12,row,col])
s=0
for year in label_year:
    print(year)
    PoPr=data_filtering(np.array(tif.imread(year+"/PrePost/PrePost*.tif")),-1500,1500,1550)
    label_arr[s,]=PoPr[:row,:col]
    s=s+1
    

In [ ]:
print(label_arr.shape)

In [ ]:
#initialize wandb and download dataset
epoch=10
batch_size=32
height=row
width=col
data=data_set.reshape([12,row,col, 4*6])
label=np.expand_dims(label_arr[:row,:col],axis=3)
train_x=data[:9]
test_x=data[9:]
train_y=label[:9]
test_y=label[9:]
print(train_x.shape)

In [ ]:
# Conv2DLSTM 
from tensorflow import keras as keras
from keras.layers import Lambda, Reshape, Permute, Input, add, Conv3D, GaussianNoise, concatenate,Conv2D
from keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Add,MaxPooling2D,UpSampling2D
from keras.models import Model

c=20

inp = Input((height, width, 4 * 6))
reshaped = Reshape((row,col,4,6))(inp)
permuted = Permute((4,1,2,3))(reshaped)
x =(ConvLSTM2D(filters=c, kernel_size=(3,3),padding='same',name='conv_lstm1', return_sequences=True))(permuted)

c1=(BatchNormalization())(x)
#x = Dropout(0.2)(x)
x =(TimeDistributed(MaxPooling2D(pool_size=(2,2))))(c1)

x =(ConvLSTM2D(filters=2*c,kernel_size=(3,3),padding='same',name='conv_lstm3',return_sequences=True))(x)
c2=(BatchNormalization())(x)
#x = Dropout(0.2)(x)

x =(TimeDistributed(MaxPooling2D(pool_size=(2,2))))(c2)
x =(ConvLSTM2D(filters=4*c,kernel_size=(3,3),padding='same',name='conv_lstm4',return_sequences=True))(x)

x =(TimeDistributed(UpSampling2D(size=(2, 2))))(x)
x =(ConvLSTM2D(filters=4*c,kernel_size=(3,3),padding='same',name='conv_lstm5',return_sequences=True))(x)
x =(BatchNormalization())(x)

x =(ConvLSTM2D(filters=2*c,kernel_size=(3,3),padding='same',name='conv_lstm6',return_sequences=True))(x)
x =(BatchNormalization())(x)
x = concatenate([c2, x])
#x = Dropout(0.2)(x)

x =(TimeDistributed(UpSampling2D(size=(2, 2))))(x)
x = concatenate([c1, x])
x =(ConvLSTM2D(filters=c,kernel_size=(3,3),padding='same',name='conv_lstm7',return_sequences=False))(x)
x =(BatchNormalization())(x)
output = Conv2D(1, (1,1))(x)
model=Model(inputs=[inp], outputs=[output])
#model.summary()
model.compile(optimizer='adam', loss='mse',metrics=[tf.keras.metrics.MeanAbsoluteError()])
          #    metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.fit(train_x,train_y,epochs=100, batch_size=2,verbose=1)

Make Prediction

In [ ]:
test=np.expand_dims(test_x[1],axis=0)

pred=model.predict(test)

In [ ]:
results = model.evaluate(test_x, test_y)
print(results)

In [ ]:
pred_show=np.squeeze(pred,axis=0)

plt.subplot(1,2,1)
plt.imshow(pred_show)
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(train_y[0])
plt.colorbar()


In [ ]:
print(train_y[0].shape)
plt.hist(train[])